# Problématique

SNCF-Transilien, l'opérateur des trains de banlieue d'Île-de-France, cherche à mieux anticiper l'augmentation du nombre de validations de cartes à puce par jour et par gare. Cette anticipation est cruciale pour améliorer la performance opérationnelle et proposer des services plus adaptés. Entre 2015 et 2019, le nombre de validations a augmenté d'environ 6% par an. Le but de ce challenge est de prédire le nombre de validations par jour et par gare à moyen et long terme.

## Objectif


L'objectif principal est de développer un modèle de machine learning capable de prédire le nombre de validations quotidiennes pour chaque gare du réseau SNCF-Transilien sur la période du 1er janvier 2023 au 30 juin 2023.

## 1. Importation et Prétraitement des Données

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
import shap

# Importation des fichiers de données
train_data = pd.read_csv('data/train_f_x.csv')
test_data = pd.read_csv('data/test_f_x_THurtzP.csv')
y_train = pd.read_csv('data/y_train_sncf.csv')

# Ajouter une colonne 'index' dans train_data et y_train pour correspondre
train_data['index'] = train_data['date'].astype(str) + '_' + train_data['station']
test_data['index'] = test_data['date'].astype(str) + '_' + test_data['station']

# Fusionner les données de validation avec les données de train
train_data = train_data.merge(y_train, on='index', how='left')

# Conversion des colonnes 'date' en format datetime
train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

# Ajout de variables exogènes
train_data['day_of_week'] = train_data['date'].dt.dayofweek
test_data['day_of_week'] = test_data['date'].dt.dayofweek
train_data['month'] = train_data['date'].dt.month
test_data['month'] = test_data['date'].dt.month
train_data['weekofyear'] = train_data['date'].dt.isocalendar().week
test_data['weekofyear'] = test_data['date'].dt.isocalendar().week

# Filtrer les stations avec au moins deux ans de données
sufficient_data_stations = train_data.groupby('station').filter(lambda x: len(x) >= 2 * 365)['station'].unique()
train_data = train_data[train_data['station'].isin(sufficient_data_stations)]
test_data = test_data[test_data['station'].isin(sufficient_data_stations)]

# Normalisation des données
features = ['job', 'ferie', 'vacances', 'day_of_week', 'month', 'weekofyear']
scaler = StandardScaler()
train_data[features] = scaler.fit_transform(train_data[features])
test_data[features] = scaler.transform(test_data[features])

# Séparer en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(train_data[features], train_data['y'], test_size=0.2, random_state=42)

# Réduction de dimensionnalité avec PCA
pca = PCA(n_components=2)
train_pca = pca.fit_transform(train_data[features])
test_pca = pca.transform(test_data[features])

# Ajout des composantes principales aux données
train_data['pca1'] = train_pca[:, 0]
train_data['pca2'] = train_pca[:, 1]
test_data['pca1'] = test_pca[:, 0]
test_data['pca2'] = test_pca[:, 1]

# Échantillonnage des données pour éviter MemoryError
train_sample = train_data.sample(n=10000, random_state=42)




c:\Users\naiml\Desktop\M272\ML\Machine-Learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Modèle de Référence : Régression Logistique

In [2]:
# Modèle de régression logistique
logistic_regression = LogisticRegression(max_iter=3)
logistic_regression.fit(X_train, y_train)

# Prédictions sur l'ensemble de validation
val_predictions_logistic = logistic_regression.predict(X_val)
mape_logistic_val = mean_absolute_percentage_error(y_val, val_predictions_logistic)
print(f'MAPE Régression Logistique (Validation): {mape_logistic_val}')

# Prédictions sur l'ensemble de test
test_predictions_logistic = logistic_regression.predict(test_data[features])
test_data['logistic_pred'] = test_predictions_logistic


MemoryError: Unable to allocate 284. GiB for an array with shape (983264, 38726) and data type float64

## 3. Modèle Non Supervisé : K-means

In [ ]:
# Clustering avec K-means
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(train_data[['pca1', 'pca2']])

# Ajouter les clusters aux données
train_data['cluster'] = kmeans.labels_
test_data['cluster'] = kmeans.predict(test_data[['pca1', 'pca2']])

# Afficher les clusters
plt.scatter(train_data['pca1'], train_data['pca2'], c=train_data['cluster'], cmap='viridis')
plt.title('Clustering avec K-means')
plt.show()


Description des Clusters :

Cluster 0 (Violet) : Composé de points situés principalement en bas à gauche. Ce cluster pourrait représenter des stations avec des caractéristiques spécifiques, comme des taux de validations plus faibles ou des particularités saisonnières distinctes.

Cluster 1 (Jaune) : Situé principalement en haut à droite. Les stations de ce cluster pourraient avoir des taux de validation plus élevés ou des variations plus régulières.

Cluster 2 (Vert) : Situé en bas à droite. Les stations ici peuvent partager des caractéristiques intermédiaires entre les clusters 0 et 1.

Cluster 3 (Bleu) : En haut à gauche. Semblable au cluster 1 mais avec des différences distinctes, peut-être en termes de variations saisonnières ou d'autres facteurs exogènes.

## 4. Modèle Supervisé : SVM avec Validation Croisée

In [ ]:
# Définir les hyperparamètres pour le SVM
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

# Initialiser le modèle SVM
svm = SVR()

# Appliquer GridSearchCV pour trouver les meilleurs hyperparamètres
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='neg_mean_absolute_percentage_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres
best_svm = grid_search.best_estimator_
print("Best hyperparameters:", grid_search.best_params_)

# Prédictions avec le SVM optimisé sur l'ensemble de validation
val_predictions_svm = best_svm.predict(X_val)
mape_svm_val = mean_absolute_percentage_error(y_val, val_predictions_svm)
print(f'MAPE SVM (Validation): {mape_svm_val}')

# Prédictions avec le SVM optimisé sur l'ensemble de test
test_predictions_svm = best_svm.predict(test_data[features])
test_data['svm_pred'] = test_predictions_svm


## 5. Interprétation des Modèles avec SHAP

In [ ]:
# Utiliser shap.sample pour réduire l'échantillon des données d'arrière-plan
background_sample = shap.sample(train_sample[['pca1', 'pca2']], 3)

# Calcul des valeurs SHAP pour le modèle SVM
explainer = shap.KernelExplainer(best_svm.predict, background_sample)
shap_values = explainer.shap_values(test_data[['pca1', 'pca2']])

# Visualiser les valeurs SHAP
shap.summary_plot(shap_values, test_data[['pca1', 'pca2']])


Le graphique SHAP ci-dessus montre l'impact des deux principales composantes principales (PCA), pca1 et pca2, sur les prédictions du modèle SVM. Les valeurs SHAP indiquent dans quelle mesure chaque variable a influencé les prédictions. Une valeur SHAP positive augmente la prédiction, tandis qu'une valeur négative la diminue. La couleur des points indique la valeur de la variable correspondante, du bleu (valeur faible) au rouge (valeur élevée).



## 6. Comparaison des Performances

In [ ]:
# Calculer la MAPE pour les modèles sur les données de test
mape_logistic_test = mean_absolute_percentage_error(test_data['y'], test_data['logistic_pred'])
print(f'MAPE Régression Logistique (Test): {mape_logistic_test}')

mape_svm_test = mean_absolute_percentage_error(test_data['y'], test_data['svm_pred'])
print(f'MAPE SVM (Test): {mape_svm_test}')


In [ ]:
# Fonction pour tracer les prévisions pour une station spécifique
def plot_predictions_per_station(station, real, logistic, svm, dates, title):
    plt.figure(figsize=(14, 7))
    plt.plot(dates, real, label='Données Réelles', color='black')
    plt.plot(dates, logistic, label='Régression Logistique', linestyle='dashed')
    plt.plot(dates, svm, label='SVM', linestyle='dashed')
    plt.xlabel('Date')
    plt.ylabel('Nombre de Validations')
    plt.title(f'{title} - Station {station}')
    plt.legend()
    plt.show()

# Sélectionner quelques stations pour afficher les prévisions
stations_to_plot = test_data['station'].unique()[:5]  # Vous pouvez ajuster le nombre de stations à afficher

for station in stations_to_plot:
    station_data = test_data[test_data['station'] == station]
    real_values = station_data['y']
    logistic_predictions = station_data['logistic_pred']
    svm_predictions = station_data['svm_pred']
    dates = station_data['date']
    
    plot_predictions_per_station(station, real_values, logistic_predictions, svm_predictions, dates, 'Comparaison des Prévisions des Modèles')
